<a href="https://colab.research.google.com/github/darvesh-sd/Copy-of-TPSessions.ipynb/blob/main/TP_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Libraries and Upload Data**

In [2]:
# Import required libraries
import pandas as pd
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')

# Specify the path to the CSV file in your Google Drive
file_path = '/content/drive/MyDrive/car_production_report.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)
print("Data loaded successfully! Here are the first few rows:")
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data loaded successfully! Here are the first few rows:
  Production ID    Car Model Production Date  Units Produced  \
0         P1000  Convertible      2023-11-24           499.0   
1         P1001          SUV      2023-02-27           132.0   
2         P1002        Truck      2023-01-13           311.0   
3         P1003        Truck      2023-05-21            98.0   
4         P1004  Convertible      2023-05-06            74.0   

   Defects Reported    Inspector Factory Location  Production Cost  
0               0.0   Jane Smith          Beijing         52258.67  
1               0.0     John Doe            Tokyo         35271.36  
2               1.0     John Doe            Tokyo         64307.05  
3               0.0   Jane Smith          Detroit         24569.88  
4               NaN  Alice Brown            Tokyo         39393.00  


**Get a Quick Overview of the Dataset**

In [3]:
# Basic dataset info
print("Dataset Overview:")
df.info()

# Display the first 5 rows for a quick glance at the data structure
print("\nFirst 5 rows of the dataset:")
print(df.head())

# Missing values check
print("\nMissing values in each column:")
print(df.isnull().sum())


Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Production ID     105 non-null    object 
 1   Car Model         105 non-null    object 
 2   Production Date   105 non-null    object 
 3   Units Produced    99 non-null     float64
 4   Defects Reported  103 non-null    float64
 5   Inspector         105 non-null    object 
 6   Factory Location  105 non-null    object 
 7   Production Cost   105 non-null    float64
dtypes: float64(3), object(5)
memory usage: 6.7+ KB

First 5 rows of the dataset:
  Production ID    Car Model Production Date  Units Produced  \
0         P1000  Convertible      2023-11-24           499.0   
1         P1001          SUV      2023-02-27           132.0   
2         P1002        Truck      2023-01-13           311.0   
3         P1003        Truck      2023-05-21            98.0   
4     

**Drop Duplicate Rows**

In [4]:
# Drop duplicates and show the difference in row count
initial_count = len(df)
df.drop_duplicates(inplace=True)
final_count = len(df)
print(f"\nNumber of duplicate rows removed: {initial_count - final_count}")
print("Updated data shape:", df.shape)



Number of duplicate rows removed: 5
Updated data shape: (100, 8)


**Fill Missing Values**

In [5]:
# Fill missing values in numerical columns with median
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Fill missing values in categorical columns with mode
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

# Verify missing values are handled
print("\nMissing values after filling:")
print(df.isnull().sum())



Missing values after filling:
Production ID       0
Car Model           0
Production Date     0
Units Produced      0
Defects Reported    0
Inspector           0
Factory Location    0
Production Cost     0
dtype: int64


**Convert Data Types (e.g., Dates)**

In [6]:
# Convert 'date' column to datetime if present
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    print("\n'Converted 'date' column to datetime format.")

# Check updated data types
print("\nData types after conversion:")
print(df.dtypes)



Data types after conversion:
Production ID        object
Car Model            object
Production Date      object
Units Produced      float64
Defects Reported    float64
Inspector            object
Factory Location     object
Production Cost     float64
dtype: object


**Normalize Text Columns**

In [7]:
# Convert all string columns to lowercase for consistency
for col in cat_cols:
    df[col] = df[col].str.lower()

print("\nFirst 5 rows after normalizing text columns:")
print(df[cat_cols].head())



First 5 rows after normalizing text columns:
  Production ID    Car Model Production Date    Inspector Factory Location
0         p1000  convertible      2023-11-24   jane smith          beijing
1         p1001          suv      2023-02-27     john doe            tokyo
2         p1002        truck      2023-01-13     john doe            tokyo
3         p1003        truck      2023-05-21   jane smith          detroit
4         p1004  convertible      2023-05-06  alice brown            tokyo


**Identify and Handle Outliers**

In [8]:
from scipy import stats
import numpy as np

# Remove rows where Z-scores are greater than 3 (indicating outliers)
initial_shape = df.shape
df = df[(np.abs(stats.zscore(df[num_cols])) < 3).all(axis=1)]
print(f"\nRemoved outliers. Row count changed from {initial_shape[0]} to {df.shape[0]}.")



Removed outliers. Row count changed from 100 to 100.


**Rename Columns for Consistency**

In [9]:
# Rename columns to be lowercase and replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_').str.lower()
print("\nUpdated column names:")
print(df.columns)



Updated column names:
Index(['production_id', 'car_model', 'production_date', 'units_produced',
       'defects_reported', 'inspector', 'factory_location', 'production_cost'],
      dtype='object')


**Create New Features**

In [10]:
# If a 'date' column exists, create a 'year' column
if 'date' in df.columns:
    df['year'] = df['date'].dt.year
    print("\nCreated 'year' column from 'date' column:")
    print(df[['date', 'year']].head())


**Drop Unnecessary Columns**

In [11]:
# Drop columns with a single unique value
columns_dropped = []
for col in df.columns:
    if df[col].nunique() == 1:
        columns_dropped.append(col)
        df.drop(columns=[col], inplace=True)

print("\nColumns dropped due to a single unique value:", columns_dropped)
print("\nRemaining columns:")
print(df.columns)



Columns dropped due to a single unique value: []

Remaining columns:
Index(['production_id', 'car_model', 'production_date', 'units_produced',
       'defects_reported', 'inspector', 'factory_location', 'production_cost'],
      dtype='object')


**Final Check and Summary**

In [12]:
# Final overview of the cleaned dataset
print("\nFinal Dataset Information:")
df.info()

# Display summary statistics
print("\nSummary statistics for numerical columns:")
print(df.describe(include=['float64', 'int64']))

print("\nFirst 5 rows of the final cleaned dataset:")
print(df.head())



Final Dataset Information:
<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   production_id     100 non-null    object 
 1   car_model         100 non-null    object 
 2   production_date   100 non-null    object 
 3   units_produced    100 non-null    float64
 4   defects_reported  100 non-null    float64
 5   inspector         100 non-null    object 
 6   factory_location  100 non-null    object 
 7   production_cost   100 non-null    float64
dtypes: float64(3), object(5)
memory usage: 7.0+ KB

Summary statistics for numerical columns:
       units_produced  defects_reported  production_cost
count      100.000000        100.000000        100.00000
mean       246.160000          1.160000      51035.43260
std        142.251003          1.502321      18768.47513
min          1.000000          0.000000      21055.53000
25%        132.750000   

In [13]:
from google.colab import files

# Save the cleaned DataFrame to a new CSV file
cleaned_file_path = '/content/cleaned_car_production_report.csv'
df.to_csv(cleaned_file_path, index=False)

# Download the file
files.download(cleaned_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>